<h1 style="font-family: Trebuchet MS; font-size: 14px; color: #9e2a2b; text-align: right; ">Created By: Muhammad Faarisul Ilmi</h1>

<h1 style="font-family: Trebuchet MS; padding: 12px; font-size: 30px; color: #720026; text-align: center; line-height: 1.25;">Time Series Analysis<br><span style="color: #ff7f51; font-size: 48px"><b>💥Hourly Energy Consumption⚡️</b></span><br><span style="color: #ff7f51; font-size: 20px">💡Using 5 Different Models</span></h1>
<hr>

<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #f7b538; text-align: center; "><b>If you find this notebook useful, give it a thumbs up 😉👍🏻</b></h1>

<p style="background-color:#720026;font-family:Trebuchet MS;font-weight:bold;color:#ff7f51;font-size:40px;text-align:center;border-radius:100px 100px">Table of Contents</p>

In this notebook, we will cover:
* [Overview](#0)
* [Importing Libraries](#1)
* [Loading Dataset](#2)
* [Data Transformation and EDA](#3)
* [Building Models](#4)
    1. [Prophet](#5)
    2. [XGBoost](#6)
    3. [Deep Neaural Network](#7)

<a id="0"></a>
<p style="background-color:#720026;font-family:Trebuchet MS;font-weight:bold;color:#ff7f51;font-size:40px;text-align:center;border-radius:100px 100px">Overview</p>

![img](https://energyoptusa.com/wp-content/uploads/2019/10/bigstock-High-Voltage-Electric-Pole-And-299162803.jpg)

PJM Interconnection LLC (PJM) is a regional transmission organization (RTO) in the United States. It is part of the Eastern Interconnection grid operating an electric transmission system serving all or parts of Delaware, Illinois, Indiana, Kentucky, Maryland, Michigan, New Jersey, North Carolina, Ohio, Pennsylvania, Tennessee, Virginia, West Virginia, and the District of Columbia

This dataset contains over 10 years of hourly energy consumption data from PJM in Megawatts

This notebook will go through the model solution for univariate time series. There are endless models out there when it comes to time series, one of the most popular methods is Prophet developed by Facebook/Meta. So, we will take a look later at building models section. This time, not only that. we will use with 5 different approaches to see which one has the best performance

<a id="1"></a>
<p style="background-color:#720026;font-family:Trebuchet MS;font-weight:bold;color:#ff7f51;font-size:40px;text-align:center;border-radius:100px 100px">Importing Libraries</p>

In [ ]:
!pip install prophet

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from prophet import Prophet
from xgboost import XGBRegressor

<a id="2"></a>
<p style="background-color:#720026;font-family:Trebuchet MS;font-weight:bold;color:#ff7f51;font-size:40px;text-align:center;border-radius:100px 100px">Loading Dataset</p>

For this project, we are going to use just 3 of the dataset to get a sense of what it looks like as the approach for every one of them will be the same

In [ ]:
df1 = pd.read_csv('../input/hourly-energy-consumption/AEP_hourly.csv', index_col='Datetime')
df2 = pd.read_csv('../input/hourly-energy-consumption/COMED_hourly.csv', index_col='Datetime')
df3 = pd.read_csv('../input/hourly-energy-consumption/DAYTON_hourly.csv', index_col='Datetime')
df4 = pd.read_csv('../input/hourly-energy-consumption/DEOK_hourly.csv', index_col='Datetime')
df5 = pd.read_csv('../input/hourly-energy-consumption/DOM_hourly.csv', index_col='Datetime')
df6 = pd.read_csv('../input/hourly-energy-consumption/DUQ_hourly.csv', index_col='Datetime')
df7 = pd.read_csv('../input/hourly-energy-consumption/EKPC_hourly.csv', index_col='Datetime')
df8 = pd.read_csv('../input/hourly-energy-consumption/FE_hourly.csv', index_col='Datetime')
df9 = pd.read_csv('../input/hourly-energy-consumption/NI_hourly.csv', index_col='Datetime')
df10 = pd.read_csv('../input/hourly-energy-consumption/PJME_hourly.csv', index_col='Datetime')
df11 = pd.read_csv('../input/hourly-energy-consumption/PJMW_hourly.csv', index_col='Datetime')

# For visualization purpose
df_final = df1.join([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11])

# We only use DAYTON_hourly.csv for modeling later since the approach is the same for each file
df = df3

In [ ]:
df_final.info()

In [ ]:
df_final.describe()

<a id="3"></a>
<p style="background-color:#720026;font-family:Trebuchet MS;font-weight:bold;color:#ff7f51;font-size:40px;text-align:center;border-radius:100px 100px">Data Transformation and EDA</p>

In [ ]:
df_final.index = pd.to_datetime(df_final.index)

df_final.plot(figsize=(20,8))
plt.title('PJM Energy Cosumption', weight='bold', fontsize=25)

That's absolutely mess. However, we can see at the first glance that `PJME` has the highest energy consumption among the rest and all the column don't share the same datetime like `NI`, `EKPC`, and `DEOK`

Let's narrow it down to the first week of 2018 to see the graph more clearly

In [ ]:
df_2018 = df_final[(df_final.index >= '2018-01-01') & (df_final.index < '2018-01-08')]

df_2018.plot(figsize=(20,8), lw=4)
plt.title('PJM Energy Cosumption\nfrom 2018-01-01 to 2018-01-08', weight='bold', fontsize=25)

In [ ]:
df_final['Hour'] = df_final.index.hour
df_final['Day'] = df_final.index.day
df_final['Month'] = df_final.index.month
df_final['Quarter'] = df_final.index.quarter
df_final['Year'] = df_final.index.year

df_final.head(3)

In [ ]:
columns = ['AEP_MW', 'COMED_MW', 'DAYTON_MW', 'DEOK_MW', 'DOM_MW', 'DUQ_MW',
           'EKPC_MW', 'FE_MW', 'NI_MW', 'PJME_MW', 'PJMW_MW']

f, axes = plt.subplots(nrows=6, ncols=2, figsize=(20, 14))
f.suptitle('Daily Average Energy Consumption', weight='bold', fontsize=25)
# We just need 11 figures, so we delete the last one
f.delaxes(axes[5][1])

for i, col in enumerate(columns):
    sns.boxplot(data=df_final, x='Hour', y=col, ax=axes.flatten()[i], color='#cc444b')

In [ ]:
f, axes = plt.subplots(nrows=6, ncols=2, figsize=(20, 14))
f.suptitle('Monthly Average Energy Consumption', weight='bold', fontsize=25)
# We just need 11 figures, so we delete the last one
f.delaxes(axes[5][1])

for i, col in enumerate(columns):
    sns.stripplot(data=df_final, x='Day', y=col, ax=axes.flatten()[i], color='#cc444b')

In [ ]:
f, axes = plt.subplots(nrows=6, ncols=2, figsize=(20, 14))
f.suptitle('Yearly Average Energy Consumption', weight='bold', fontsize=25)
# We just need 11 figures, so we delete the last one
f.delaxes(axes[5][1])

for i, col in enumerate(columns):
    sns.barplot(data=df_final, x='Month', y=col, ax=axes.flatten()[i], color='#cc444b')

In [ ]:
f, axes = plt.subplots(nrows=6, ncols=2, figsize=(20, 14))
f.suptitle('Average Energy Consumption\nfrom 2014-2018', weight='bold', fontsize=25)
# We just need 11 figures, so we delete the last one
f.delaxes(axes[5][1])

for i, col in enumerate(columns):
    sns.violinplot(data=df_final, x='Year', y=col, ax=axes.flatten()[i], color='#cc444b')

Our data have seasonality. That's why all visualization are pretty much the same for all the data or columns except for the year which some data have missing value in certain year

What are the insights we got from above EDA?
* `PJME` holds the highest energy consumption among others
* Daily peak/highest is around 6-8 PM and the trough/mininum is at 2 AM
* The highest energy consumption in a year is either in the end of the year or in the middle of the year
* Out of 11 data, 5 of them is not complete (2004-2018)

<h1 style="font-family: Trebuchet MS; font-size: 25px; color: #da627d; text-align: left; "><b>DATASET: DAYTON_MW</b></h1>

The actual column or data that we use in this notebook is from `DAYTON_MW` to simplify the process beccause the rest of the data will have the exact same approach as DAYTON

In [ ]:
# Set color style
plt.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=['#cc444b', '#e89005'])

In [ ]:
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# The data that we are going to use (DAYTON)
df.head()

In [ ]:
seasonal_decompose(df, period=365).plot()
plt.show()

<h1 style="font-family: Trebuchet MS; font-size: 25px; color: #da627d; text-align: left; "><b>Moving Average</b></h1>

In [ ]:
# Daily Consumption
ma = df.resample('D').mean()

# 3 Day Example
ma['Moving Average'] = ma['DAYTON_MW'].rolling(3).mean()
ma.head()

In [ ]:
def moving_average(data, window):
    data['Moving Average'] = data['DAYTON_MW'].rolling(window).mean()
    actual = data['DAYTON_MW'][-(window+30):]
    ma = data['Moving Average'][-(window+30):]
    
    plt.figure(figsize=(20,8))
    actual.plot(label='Actual', lw=4)
    ma.plot(label='MA-{}'.format(str(window)), ls='--', lw=2)
    plt.title('{}-Days Moving Average'.format(str(window)), weight='bold', fontsize=25)
    plt.legend()

In [ ]:
moving_average(ma, 7)

In [ ]:
moving_average(ma, 30)

In [ ]:
# Before building and training our model, let's split the data into training and testing
df_train, df_test = df[df.index < '2016-01-01'], df[df.index >= '2016-01-01']

print('Train:\t', len(df_train))
print('Test:\t', len(df_test))

In [ ]:
plt.figure(figsize=(20,8))

df_train['DAYTON_MW'].plot(label='Training Set')
df_test['DAYTON_MW'].plot(label='Test Set')
plt.axvline('2016-01-01', color='black', ls='--', lw=3)
plt.text('2016-02-01', 3700, 'Split', fontsize=20, fontweight='bold')
plt.title('Data Splitting', weight='bold', fontsize=25)
plt.legend()

<a id="4"></a>
<p style="background-color:#720026;font-family:Trebuchet MS;font-weight:bold;color:#ff7f51;font-size:40px;text-align:center;border-radius:100px 100px">Building Models</p>

<a id="5"></a>
<h1 style="font-family: Trebuchet MS; font-size: 25px; color: #da627d; text-align: left; "><b>1. Prophet</b></h1>

Prophet is an open-source library for univariate (one variable) time series forecasting developed by Facebook. It works best with time series that have strong seasonal effects and several seasons of historical data

Official Documentation: [Here](http://facebook.github.io/prophet/#:~:text=Prophet%20is%20a%20procedure%20for,several%20seasons%20of%20historical%20data.)

Helpful Resources: 
* https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/
* https://www.kaggle.com/code/prashant111/tutorial-time-series-forecasting-with-prophet/notebook

In [ ]:
def index_to_column(data):
    data = data.reset_index()
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    data = data.sort_values('Datetime')
    
    data = data.rename(columns={'Datetime': 'ds', 'DAYTON_MW': 'y'})
    return data

In [ ]:
prophet_train = index_to_column(df_train)
prophet_test = index_to_column(df_test)

In [ ]:
prophet_model = Prophet(interval_width=0.95)

prophet_model.fit(prophet_train)
prophet_pred = prophet_model.predict(prophet_test[['ds']]) # Keep the dataset format

In [ ]:
mae = round(mean_absolute_error(prophet_test['y'], prophet_pred['yhat']), 3)

plt.figure(figsize=(20,8))
plt.plot(prophet_test['ds'], prophet_test['y'], label='Actual')
plt.plot(prophet_pred['ds'], prophet_pred['yhat'], label='Predicted')
plt.title('Test Forecasting', weight='bold', fontsize=40)
plt.text(16770, 3250, 'MAE: {}'.format(mae), fontsize=20, color='red')
plt.title('Testing Set Forecast', weight='bold', fontsize=25)
plt.legend()

<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #da627d; text-align: left; "><b>Predict the value for next 7 days</b></h1>

In [ ]:
# This time, we will use all data (train and test) to train our model
new_df = index_to_column(df)

In [ ]:
prophet_model2 = Prophet(interval_width=0.95)
prophet_model2.fit(new_df)
# 7 days to the future (7x24 = 168)
future_dates = prophet_model2.make_future_dataframe(periods=168, freq='H')
prophet_pred2 = prophet_model2.predict(future_dates)

In [ ]:
plt.figure(figsize=(20,8))

fig = prophet_model2.plot(prophet_pred2, uncertainty=True)
ax = fig.gca()
ax.set_xlim(pd.to_datetime(['2018-07-01', '2018-08-15']))
plt.title('7 Days Forecast', weight='bold', fontsize=25)
plt.show()

In [ ]:
prophet_model.plot_components(prophet_pred2)
plt.show()

<a id="6"></a>
<h1 style="font-family: Trebuchet MS; font-size: 25px; color: #da627d; text-align: left; "><b>2. XGBoost</b></h1>

XGBoost is short for Extreme Gradient Boosting and is an efficient implementation of the stochastic gradient boosting machine learning algorithm

The stochastic gradient boosting algorithm, also called gradient boosting machines or tree boosting, is a powerful machine learning technique that performs well or even best on a wide range of challenging machine learning problems

To be able to use XGBoost for time series forecasting, the data should be transformed into supervised learning before feeding it into the model

Official Documentation: [here](https://xgboost.readthedocs.io/en/stable/python/python_api.html)

Helpful Resources:
* https://machinelearningmastery.com/xgboost-for-time-series-forecasting/
* https://www.kaggle.com/code/robikscube/tutorial-time-series-forecasting-with-xgboost

In [ ]:
def date_transform(data):
    df = data.copy()
    
    df['Hour'] = df.index.hour
    df['Dayofweek'] = df.index.dayofweek
    df['Dayofmonth'] = df.index.day
    df['Dayofyear'] = df.index.dayofyear
    df['weekofyear'] = df.index.weekofyear
    df['Month'] = df.index.month
    df['Quarter'] = df.index.quarter
    df['Year'] = df.index.year
    
    X = df.drop('DAYTON_MW', axis=1)
    y = df['DAYTON_MW']
    
    return X, y

In [ ]:
X_train, y_train = date_transform(df_train)
X_test, y_test = date_transform(df_test)

In [ ]:
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, early_stopping_rounds=10)
xgb_model.fit(X_train, y_train, eval_metric='mae', eval_set=[(X_train, y_train), (X_test, y_test)])
xgb_pred = xgb_model.predict(X_test)

In [ ]:
mae = round(mean_absolute_error(y_test, xgb_pred), 3)

df_plot = pd.DataFrame({'y_test':y_test, 'xgb_pred':xgb_pred})

plt.figure(figsize=(20,8))

df_plot['y_test'].plot(label='Actual')
df_plot['xgb_pred'].plot(label='Predicted')
plt.text(16770, 3250, 'MAE: {}'.format(mae), fontsize=20, color='red')
plt.title('Testing Set Forecast', weight='bold', fontsize=25)
plt.legend()
plt.show()

<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #da627d; text-align: left; "><b>Predict the value for next 7 days</b></h1>

In [ ]:
future_dates2 = future_dates.iloc[-168:, :].copy()

future_dates2['ds'] = pd.to_datetime(future_dates2['ds'])
future_dates2 = future_dates2.set_index('ds')

future_dates2['Hour'] = future_dates2.index.hour
future_dates2['Dayofweek'] = future_dates2.index.dayofweek
future_dates2['Dayofmonth'] = future_dates2.index.day
future_dates2['Dayofyear'] = future_dates2.index.dayofyear
future_dates2['weekofyear'] = future_dates2.index.weekofyear
future_dates2['Month'] = future_dates2.index.month
future_dates2['Quarter'] = future_dates2.index.quarter
future_dates2['Year'] = future_dates2.index.year

In [ ]:
X = pd.concat([X_train, X_test], ignore_index=True)
y = pd.concat([y_train, y_test], ignore_index=True)

In [ ]:
xgb_model2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)
xgb_model2.fit(X, y, eval_metric='mae')
xgb_pred2 = xgb_model2.predict(future_dates2)

In [ ]:
df_plot2 = pd.DataFrame({'Hour':future_dates2['Hour'], 'xgb_pred2':xgb_pred2})

last_week = df['2018-07-01':'2018-08-15']

plt.figure(figsize=(20,8))

last_week['DAYTON_MW'].plot()
df_plot2['xgb_pred2'].plot()
plt.title('7 Days Forecast', weight='bold', fontsize=25)
plt.show()

<a id="7"></a>
<h1 style="font-family: Trebuchet MS; font-size: 25px; color: #da627d; text-align: left; "><b>3. Deep Neaural Network</b></h1>

The Deep Learning framework we are using is Tensorflow

<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #da627d; text-align: left; "><b>Preparing features and labels</b></h1>

Before feeding the data into Neural Network, we have to do some modification to the data so they can be accepted by the model. We are going to use windowing technique which basically group the data into feature and label. The label will be the next value. You can take a look at the next few cells to give an idea what we are going to do

Official Documentation: [Here](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

Helpful Resource:

* https://stackoverflow.com/questions/55429307/how-to-use-windows-created-by-the-dataset-window-method-in-tensorflow-2-0

In [ ]:
dataset = tf.expand_dims(df_train['DAYTON_MW'].head(10), axis=-1)

# Generate a tf dataset with 10 elements (i.e. numbers 0 to 9)
dataset = tf.data.Dataset.from_tensor_slices(dataset)

# Window the data but only take those with the specified size
dataset = dataset.window(5, shift=1, drop_remainder=True)

# Flatten the windows by putting its elements in a single batch
dataset = dataset.flat_map(lambda window: window.batch(5))

# Create tuples with features (first four elements of the window) and labels (last element)
dataset = dataset.map(lambda window: (window[:-1], window[-1]))

# # Shuffle the windows
# dataset = dataset.shuffle(buffer_size=10)

# # Create batches of windows
# dataset = dataset.batch(2).prefetch(1)

# Print the results
for x,y in dataset:
    print("x = ", x.numpy())
    print("y = ", y.numpy())
    print()

<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #da627d; text-align: left; "><b>Build and train the model</b></h1>

In [ ]:
def windowing(data, window_size, shuffle_buffer, batch_size):
    dataset = tf.expand_dims(data, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices(dataset)
    dataset = dataset.window(window_size+1, shift=1, drop_remainder=True) # window size = 24 + 1 (test)
    dataset = dataset.flat_map(lambda window: window.batch(window_size+1))
    dataset = dataset.map(lambda window: (window[:-1], window[-1])) # (train, test) 
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

In [ ]:
train = windowing(df_train['DAYTON_MW'], 24, 72, 32)
test = windowing(df_test['DAYTON_MW'], 24, 72, 32)

In [ ]:
dnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=16, kernel_size=3,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[24, 1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, activation='relu')),
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(1)
])

dnn_model.summary()

In [ ]:
dnn_model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
dnn_model.fit(train, validation_data=test, epochs=20)

In [ ]:
metric = pd.DataFrame(dnn_model.history.history)
metric.plot()

In [ ]:
window_size = 24
forecast = []

train_length = len(df_train)
forecast_series = df[train_length - window_size:]

# Use the model to predict data points per window size
for time in range(len(forecast_series) - window_size):
    forecast.append(dnn_model.predict(np.expand_dims(forecast_series[time:time + window_size], axis=-1)[np.newaxis]))

# Convert to a numpy array and drop single dimensional axes
results = np.array(forecast).squeeze()

In [ ]:
mae = round(mean_absolute_error(df_test['DAYTON_MW'], df_test['Pred']), 3)

df_test['Pred'] = results

plt.figure(figsize=(20,8))

df_test['DAYTON_MW'].plot(label='Actual')
df_test['Pred'].plot(label='Predicted')
plt.text(16770, 3250, 'MAE: {}'.format(mae), fontsize=20, color='red')
plt.title('Testing Set Forecast', weight='bold', fontsize=25)
plt.legend()
plt.show()

<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #f7b538; text-align: center; "><b>If you find this notebook useful, give it a thumbs up 😉👍🏻</b></h1>
<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #fb8500; text-align: center; "><b>If you have any feedbacks or suggestions on how to improve model performance, please let me know in the comment</b></h1>
<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #fb8500; text-align: center; "><b>THANK YOU</b></h1>

<h1 style="font-family: Trebuchet MS; font-size: 60px; color: #9e2a2b; text-align: center;"><b>THE END</b></h1>

<h1 style="font-family: Trebuchet MS; font-size: 14px; color: #9e2a2b; text-align: right; ">Created By: Muhammad Faarisul Ilmi</h1>